# Transformed Hourly Weather Data
Author: Colin Pannikkat

This notebook transforms the Posch et. al hourly weather data into a usable input file for the GARISOM model. Soil temperature was not provided, and is instead retrieved from NLDAS in file_builder.py when building the simulation files. 

In [2]:
import pandas as pd
from datetime import datetime
import numpy as np

In [3]:
hourly_weather = pd.read_csv("./data/weather_hourly.avg_230501-231231.csv")
hourly_weather.head()

,No.,Date.Time,Rain.mm,Wind.Direction,Temp.C,RH.percent,Wind.Speed.m.s,Gust.Speed.m.s,PAR.mumol.m2.s
0,26333.5,5/1/2023 0:43,0.0,110.95,25.2760,17.60,0.00,1.260,1.2
1,26335.5,5/1/2023 1:43,0.0,57.60,23.2815,21.90,0.00,0.755,1.2
2,26337.5,5/1/2023 2:43,0.0,68.10,23.2805,23.40,0.00,1.260,1.2
3,26339.5,5/1/2023 3:43,0.0,63.20,21.7000,25.25,0.25,1.760,1.2
4,26341.5,5/1/2023 4:43,0.0,44.20,21.1990,25.55,0.50,2.010,1.2


In [4]:
water_amount = pd.read_csv("./data/water amounts_pots_2023.csv")
water_amount.head()

,Date.yymmdd,DOY,Time.Start,Time.Stop,irrigation.mm3,irrigation.mm,precipitation.mm
0,230527,147,0:00:00,0:01:00,0.0,0.0,0.0
1,230527,147,0:01:00,0:02:00,0.0,0.0,0.0
2,230527,147,0:02:00,0:03:00,0.0,0.0,0.0
3,230527,147,0:03:00,0:04:00,0.0,0.0,0.0
4,230527,147,0:04:00,0:05:00,0.0,0.0,0.0


In [5]:
water_amount['Year'] = water_amount['Date.yymmdd'].map(lambda x: datetime.strptime(str(x), "%y%m%d").strftime("%Y"))

In [6]:
water_amount['Day'] = water_amount['DOY'].map(lambda x: datetime.strptime(str(x), "%j").strftime("%-j"))

In [7]:
water_amount['Hour'] = water_amount['Time.Start'].map(lambda x: datetime.strptime(x, "0:%H:%M").strftime("%H"))

In [8]:
water_amount.head()

,Date.yymmdd,DOY,Time.Start,Time.Stop,irrigation.mm3,irrigation.mm,precipitation.mm,Year,Day,Hour
0,230527,147,0:00:00,0:01:00,0.0,0.0,0.0,2023,147,00
1,230527,147,0:01:00,0:02:00,0.0,0.0,0.0,2023,147,01
2,230527,147,0:02:00,0:03:00,0.0,0.0,0.0,2023,147,02
3,230527,147,0:03:00,0:04:00,0.0,0.0,0.0,2023,147,03
4,230527,147,0:04:00,0:05:00,0.0,0.0,0.0,2023,147,04


In [9]:
new_hourly_weather = pd.DataFrame(columns=['Year', 'Day', 'Hour', 'Solar_Wm2', 'Rain_mm', 'Wind_ms.1', 'Tair_C', 'D_kPa'])

In [10]:
new_hourly_weather['Rain_mm'] = hourly_weather['Rain.mm']

In [11]:
new_hourly_weather['Wind_ms.1'] = hourly_weather['Wind.Speed.m.s']

In [12]:
new_hourly_weather['Tair_C'] = hourly_weather['Temp.C']

In [13]:
new_hourly_weather['Year'] = hourly_weather['Date.Time'].map(lambda x: datetime.strptime(x, "%m/%d/%Y %H:%M").strftime("%Y"))

In [14]:
new_hourly_weather['Day'] = hourly_weather['Date.Time'].map(lambda x: datetime.strptime(x, "%m/%d/%Y %H:%M").strftime("%-j"))

In [15]:
new_hourly_weather['Hour'] = hourly_weather['Date.Time'].map(lambda x: datetime.strptime(x, "%m/%d/%Y %H:%M").strftime("%H"))

In [16]:
new_hourly_weather.set_index(['Year','Day','Hour'])['Rain_mm']

Year  Day  Hour
2023  121  00      0.0
           01      0.0
           02      0.0
           03      0.0
           04      0.0
                  ... 
      365  19      0.0
           20      0.0
           21      0.0
           22      0.0
           23      0.0
Name: Rain_mm, Length: 5880, dtype: float64

In [17]:
water_amount.set_index(['Year', 'Day', 'Hour'])['irrigation.mm']

Year  Day  Hour
2023  147  00      0.0
           01      0.0
           02      0.0
           03      0.0
           04      0.0
                  ... 
      301  19      0.0
           20      0.0
           21      0.0
           22      0.0
           23      0.0
Name: irrigation.mm, Length: 3696, dtype: float64

In [18]:
# Align indices before addition
water_amount_indexed = water_amount.set_index(['Year', 'Day', 'Hour'])['irrigation.mm'] / 10
new_hourly_weather_indexed = new_hourly_weather.set_index(['Year', 'Day', 'Hour'])['Rain_mm']

# Reindex water_amount_indexed to match new_hourly_weather_indexed
water_amount_indexed = water_amount_indexed.reindex(new_hourly_weather_indexed.index, fill_value=0)

# Perform addition with aligned indices
new_hourly_weather['Rain_mm'] = water_amount_indexed.add(new_hourly_weather_indexed, fill_value=0).reset_index(drop=True)

In [19]:
# Add irrigation values for pre May 27th twice a day, 6.32mm
new_hourly_weather.loc[
    (new_hourly_weather['Day'].astype(int) < 147) & 
    ((new_hourly_weather['Hour'].astype(int) == 6) | (new_hourly_weather['Hour'].astype(int) == 18)),
    'Rain_mm'
] += 6.32

In [20]:
def calc_e_water(T):
    '''
    Calculate saturation vapor pressure for water.
    '''
    return 6.1121 * np.exp((18.678 - (T / 234.6)) * (T / (257.14 + T)))
def calc_e_ice(T):
    '''
    Calculate saturation vapor pressure for ice.
    '''
    return 6.1115 * np.exp((23.036 - (T / 333.7)) * (T / (279.824 + T)))

In [21]:
def calc_vpd(air_temp, rh, saturation_vapor_pressure):
    '''
    Calculates VPD according to saturation vapor pressure calculations of Buck 
    (1996), these are modifications of Buck (1981) that does not require an 
    enhancement factor specification.

    VPD = e_s * (1 - RH/100)
    e_s is dependent on whether T > 0 or < 0

    air_temp must be in C, rh in percent, saturation_vapor_pressure uses Buck
    calculations which returns hPa not kPa.
    '''
    return (saturation_vapor_pressure(air_temp) * (1 - (rh / 100))) * 0.1 # 1 hPa to 0.1 kPa

In [22]:
# Calculate VPD in kPa
new_hourly_weather['D_kPa'] = hourly_weather.apply(
    lambda row: calc_vpd(row['Temp.C'], row['RH.percent'], calc_e_water) if row['Temp.C'] > 0 else calc_vpd(row['Temp.C'], row['RH.percent'], calc_e_ice),
    axis=1
)

In [23]:
def convert_par_to_solar_radiation(par):
    '''
    Conversion done per:

    Reis, Mariana & Ribeiro, Aristides. (2020). Conversion factors and general 
    quations applied in agricultural and forest meteorology. 27. 227-258. 
    10.31062/agrom.v27i2.26527. 

    "The approximation 1 W m-2 ≈ 4.57 μmol m-2 s-1 (Thimijan & Heins, 1983) is 
    assuming that the W m-2 is for photosynthetically active radiation (PAR) 
    from 4.0 to 7.0 µm."

    Sensor used for cottonwood data was HOBO S-LIA-M003, which measures
    between 400 to 700 nm, so this is fine to use, but for other sensors that do
    not measure in that range, PAR is ~2.02 instead.
    '''
    return par / 4.57

In [24]:
# Subtract weird baseline (1.2) and calculate solar radiation in Wm^-2 from micromoles/m2/s
new_hourly_weather['Solar_Wm2'] = hourly_weather['PAR.mumol.m2.s'].apply(lambda x: x - 1.2).apply(convert_par_to_solar_radiation)

In [25]:
new_hourly_weather.to_csv("./dataset.csv", index=False)